In [6]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from auto_gptq import AutoGPTQForCausalLM

In [7]:

model_name_or_path = "models\TheBloke_Wizard-Vicuna-13B-Uncensored-GPTQ"


params = {'model_basename': 'Wizard-Vicuna-13B-Uncensored-GPTQ-4bit-128g.compat.no-act-order', 
          'device': 'cuda:0', 
          'use_triton': False, 
          'inject_fused_attention': True, 
          'inject_fused_mlp': True, 
          'use_safetensors': True, 
          'trust_remote_code': False, 
          'max_memory': {0: '11GiB', 'cpu': '99GiB'}, 
          'quantize_config': None, 
          'use_cuda_fp16': True}

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path, **params)

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

The safetensors archive passed at models\TheBloke_Wizard-Vicuna-13B-Uncensored-GPTQ\Wizard-Vicuna-13B-Uncensored-GPTQ-4bit-128g.compat.no-act-order.safetensors does not contain metadata. Make sure to save your model with the `save_pretrained` method. Defaulting to 'pt' metadata.
Skipping module injection for FusedLlamaMLPForQuantizedModel as currently not supported with use_triton=False.


In [10]:
HuggingFacePipeline

Init signature:
HuggingFacePipeline(
    *,
    cache: Optional[bool] = None,
    verbose: bool = None,
    callbacks: Union[List[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None,
    callback_manager: Optional[langchain_core.callbacks.base.BaseCallbackManager] = None,
    tags: Optional[List[str]] = None,
    metadata: Optional[Dict[str, Any]] = None,
    pipeline: Any = None,
    model_id: str = 'gpt2',
    model_kwargs: Optional[dict] = None,
    pipeline_kwargs: Optional[dict] = None,
    batch_size: int = 4,
) -> None
Docstring:     
HuggingFace Pipeline API.

To use, you should have the ``transformers`` python package installed.

Only supports `text-generation`, `text2text-generation` and `summarization` for now.

Example using from_model_id:
    .. code-block:: python

        from langchain_community.llms import HuggingFacePipeline
        hf = HuggingFacePipeline.from_model_id(
            model_id="gpt2",


In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "models\TheBloke_Wizard-Vicuna-13B-Uncensored-GPTQ"
# To use a different branch, change revision
# For example: revision="latest"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

prompt = "Tell me about AI"
prompt_template=f'''A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: {prompt} ASSISTANT:

'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

print(pipe(prompt_template)[0]['generated_text'])

OSError: Error no file named pytorch_model.bin, tf_model.h5, model.ckpt.index or flax_model.msgpack found in directory models\TheBloke_Wizard-Vicuna-13B-Uncensored-GPTQ.